In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
def od_ordinal(X, object_cols):
    """Takes in X dataframe and a list of column names with object
       data type and returns an ordinal encoded X dataframe."""

    ordinal = OrdinalEncoder()
    X[object_cols] = pd.DataFrame(ordinal.fit_transform(X[object_cols]))
    return X

def od_imputer(*X_tuple, numerical_cols, categorical_cols):
    """Imputes the numerical and categorical columns of X_train and/or X_test before returning the
       said dataframe(s)."""

    imputer_mean = SimpleImputer()
    imputer_mode = SimpleImputer(strategy='most_frequent')

    for x in X_tuple:
        x[numerical_cols] = imputer_mean.fit_transform(x[numerical_cols])
        x[categorical_cols] = imputer_mode.fit_transform(x[categorical_cols])
    
    return X_tuple

In [3]:
def normalize(x):
    return ' '.join([a.strip(',()[].\'"') for a in x.split(' ')])

def ticket_number(x):
    return x.split(' ')[-1]

def ticket_object(x):
    if len(x) == 1:
        return 'NONE'
    return '_'.join(x.split(' ')[:-1])

In [4]:
train_set = pd.read_csv('train.csv')

X = train_set.drop('Survived', axis=1)
y = train_set.Survived
X_test = pd.read_csv('test.csv')

X.Name = X.Name.apply(normalize)
X['Ticket_Number'] = X.Ticket.apply(ticket_number)
X.loc[X['Ticket_Number'] == 'LINE', 'Ticket_Number'] = np.nan
X['Ticket'] = X.Ticket.apply(ticket_object)

X_test.Name = X_test.Name.apply(normalize)
X_test['Ticket_Number'] = X_test.Ticket.apply(ticket_number)
X_test.loc[X_test['Ticket_Number'] == 'LINE', 'Ticket_Number'] = np.nan
X_test['Ticket'] = X_test.Ticket.apply(ticket_object)

object_cols = [col for col in X.columns if X[col].dtype=='object']
numerical_cols = list(set(X.columns) - set(object_cols))

X = od_ordinal(X, object_cols)
X_test = od_ordinal(X_test, object_cols)

In [5]:
X, X_test = od_imputer(X, X_test, numerical_cols=numerical_cols, categorical_cols=object_cols)

In [6]:
model = keras.Sequential([layers.BatchNormalization(input_shape=[12]),
                         layers.Dense(512, activation='relu'),
                         layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                         layers.Dense(512, activation='relu'),
                         layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                         layers.Dense(512, activation='relu'),
                         layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                         layers.Dense(1, activation='sigmoid')])

model.compile(loss=keras.losses.BinaryCrossentropy(),
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])

In [8]:
model.fit(X, y, epochs=70, batch_size=64)

yhat_test = np.where(model.predict(X_test).squeeze() >= 0.5, 1, 0) 

output = pd.DataFrame({'PassengerId': X_test.PassengerId.astype('int32'),
                       'Survived': yhat_test,})

output.to_csv('submission_nn.csv', index=False)

Epoch 1/80
14/14 [==============================] - 0s 14ms/step - loss: 0.2101 - accuracy: 0.9169
Epoch 2/80
14/14 [==============================] - 0s 14ms/step - loss: 0.1990 - accuracy: 0.9068
Epoch 3/80
14/14 [==============================] - 0s 13ms/step - loss: 0.2008 - accuracy: 0.9169
Epoch 4/80
14/14 [==============================] - 0s 14ms/step - loss: 0.2174 - accuracy: 0.9158
Epoch 5/80
14/14 [==============================] - 0s 14ms/step - loss: 0.2221 - accuracy: 0.9136
Epoch 6/80
14/14 [==============================] - 0s 14ms/step - loss: 0.1843 - accuracy: 0.9181
Epoch 7/80
14/14 [==============================] - 0s 15ms/step - loss: 0.2000 - accuracy: 0.9237
Epoch 8/80
14/14 [==============================] - 0s 14ms/step - loss: 0.1965 - accuracy: 0.9136
Epoch 9/80
14/14 [==============================] - 0s 15ms/step - loss: 0.1839 - accuracy: 0.9226
Epoch 10/80
14/14 [==============================] - 0s 14ms/step - loss: 0.2001 - accuracy: 0.9113
Epoch 11/